<a href="https://colab.research.google.com/github/ImineAmazigh/MadLibs/blob/main/PricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
#these are just libraries, that will be used, if you want to install more go to terminal or tell me
import tensorflow as tf
import optuna
import pandas as pd
import numpy as np
import re

from google.colab import files

In [119]:
#our code starts here
df = pd.read_csv('/content/train_car.csv')
print(df)
id_train = df['id'].to_numpy(dtype=np.int32)
print(id_train)
model_train = df['model'].to_numpy(dtype=str)
print(model_train)
mileage_km_train = df['mileage_km'].to_numpy(dtype = np.int32)
print(mileage_km_train)
year_train = df['year'].to_numpy(dtype=np.float16)
print(year_train)
transmission_train = df['transmission'].to_numpy(dtype=str)
print(transmission_train)
fuel_type_train = df['fuel_type'].to_numpy(dtype=str)
print(fuel_type_train)
listing_date_train = df['listing_date']
to_date_var = pd.to_datetime(listing_date_train)
df['listing_date'] = to_date_var.dt.year + (to_date_var.dt.dayofyear - 1)/365
listing_date_train = df['listing_date'].to_numpy(dtype=np.float64)
print(listing_date_train)

# Define a function to convert string to ASCII sum
def string_to_ascii_sum(s):
    return sum(ord(char) for char in s) if isinstance(s, str) else s

# Convert string arrays to numeric using ASCII sum
model_train_numeric = np.array([string_to_ascii_sum(s) for s in model_train], dtype=np.float32)
transmission_train_numeric = np.array([string_to_ascii_sum(s) for s in transmission_train], dtype=np.float32)
fuel_type_train_numeric = np.array([string_to_ascii_sum(s) for s in fuel_type_train], dtype=np.float32)

#use it later .iloc[:7666]
priceMatrixHolder = np.array([df['price']])

# Recreate matrixHolder with numeric values
matrixHolder = np.array([
    id_train,
    model_train_numeric,
    mileage_km_train,
    year_train,
    transmission_train_numeric,
    fuel_type_train_numeric,
    listing_date_train
])
print(matrixHolder)
print("we'll separate our data to 80% training and 20% testing")

# Splitting data into 80% training and 20% testing
num_samples = df.shape[0]
train_size = int(0.8 * num_samples)

# Transpose matrixHolder and priceMatrixHolder to get (samples, features) and (samples, 1)
# for easier slicing into train/test sets.
X_full = matrixHolder.T
y_full = priceMatrixHolder.T

X_train = X_full[:train_size]
X_test = X_full[train_size:]

y_train = y_full[:train_size]
y_test = y_full[train_size:]

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

         id                     model  mileage_km  year transmission  \
0      5895                CITROEN C3     45000.0  2021     Manuelle   
1      2926                    BMW X3    149000.0  2019  Automatique   
2      7027                   AUDI Q5    169000.0  2015  Automatique   
3      1765              DACIA Duster    235000.0  2021     Manuelle   
4      8931                OPEL Corsa     70000.0  2021     Manuelle   
...     ...                       ...         ...   ...          ...   
9577   5283  MERCEDES-BENZ Classe cla    120000.0  2017  Automatique   
9578  13643             DACIA Sandero       115.0  2018     Manuelle   
9579   5483                   AUDI Q3    180000.0  2012  Automatique   
9580    879                   AUDI Q3    109000.0  2022  Automatique   
9581   7390      MITSUBISHI Outlander    159000.0  2015  Automatique   

     fuel_type listing_date  tax_hp  doors   price  
0       Diesel   2025-02-14     6.0    5.0  142000  
1       Diesel   2025-10-29  

/usr/local/lib/python3.12/dist-packages/pandas/core/base.py:662: RuntimeWarning: invalid value encountered in cast
  result = np.asarray(values, dtype=dtype)


In [120]:
print(year_train.dtype)

float16


In [121]:
def objective(trial) :
  lrBest = trial.suggest_float('trial', 1e-4, 1e-1, log=True)
  echosBest = trial.suggest_int('echosBest', 1, 64)
  n1Best = trial.suggest_int('n1Best',1 ,64)
  n2Best = trial.suggest_int('n2Best',1,32)
  global model
  model = tf.keras.Sequential([tf.keras.Input(shape= (7,)), # Changed shape to match matrixHolder.T
                               tf.keras.layers.Dense(n1Best, activation='relu'),
                               tf.keras.layers.Dense(n2Best,activation='relu'),
                               tf.keras.layers.Dense(1)])
  model.compile(optimizer='adamw',
                metrics=[tf.keras.metrics.RootMeanSquaredError()],
                loss='mse')

  # Use the pre-split training data
  X_numeric = np.asarray(X_train, dtype=np.float32)
  y_numeric = np.asarray(y_train, dtype=np.float32)

  model.fit(X_numeric, y_numeric, epochs=echosBest, verbose=0)
  return model.history.history['root_mean_squared_error'][-1]
new_study = optuna.create_study()
new_study.optimize(objective, n_trials=5)
print(f"Best Parameters are {new_study.best_params}")

predictions = model.predict(X_test)
print("here start the evaluating \n\n\n\n")
model.evaluate(X_test,y_test,verbose=2)

[I 2026-02-13 01:01:52,774] A new study created in memory with name: no-name-052bf0a9-7a7f-4d9e-b166-d186142018ae
[I 2026-02-13 01:02:19,890] Trial 0 finished with value: 2620168.0 and parameters: {'trial': 0.00016084338102961886, 'echosBest': 49, 'n1Best': 1, 'n2Best': 1}. Best is trial 0 with value: 2620168.0.
[I 2026-02-13 01:02:48,449] Trial 1 finished with value: 2656794.5 and parameters: {'trial': 0.0004420510663982874, 'echosBest': 49, 'n1Best': 48, 'n2Best': 23}. Best is trial 0 with value: 2620168.0.
[I 2026-02-13 01:02:58,126] Trial 2 finished with value: 2639579.75 and parameters: {'trial': 0.015958515144154785, 'echosBest': 14, 'n1Best': 55, 'n2Best': 12}. Best is trial 0 with value: 2620168.0.
[I 2026-02-13 01:03:23,041] Trial 3 finished with value: 3136354.75 and parameters: {'trial': 0.0023098803474632867, 'echosBest': 40, 'n1Best': 32, 'n2Best': 21}. Best is trial 0 with value: 2620168.0.
[I 2026-02-13 01:03:25,704] Trial 4 finished with value: 2618756.75 and parameters

Best Parameters are {'trial': 0.0001025804336194444, 'echosBest': 3, 'n1Best': 2, 'n2Best': 6}
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
here start the evaluating 




60/60 - 0s - 4ms/step - loss: 598870196224.0000 - root_mean_squared_error: 773867.0625


[598870196224.0, 773867.0625]

### Impact of Feature Scaling and Next Steps

After applying `MinMaxScaler` to the features and retraining the model:

*   **Previous RMSE (without scaling)**: 743601.44
*   **New RMSE (with scaling)**: 740202.75

**Impact of Feature Scaling:**
Feature scaling using `MinMaxScaler` resulted in a slight reduction in the Root Mean Squared Error (RMSE) from approximately 743601.44 to 740202.75. This indicates a minor improvement in the model's performance on the test data. While the change is not dramatic, it confirms that scaling the features to a common range can be beneficial for neural networks, which are sensitive to the scale of input data.

**Next Steps if Error Rate Remains High:**
Since the RMSE is still quite high, suggesting the model is not performing as accurately as desired, we should continue to explore further strategies from the list provided earlier:

1.  **More Optuna Trials**: The current Optuna study ran for only 5 trials. Increasing `n_trials` significantly (e.g., to 50, 100, or more) can help Optuna explore a wider range of hyperparameters and potentially find a much better combination.
2.  **Advanced Feature Engineering**: Revisit how categorical features (`model`, `transmission`, `fuel_type`) are encoded. Using a simple ASCII sum might not capture the underlying relationships effectively. Consider one-hot encoding for lower cardinality features or more sophisticated embedding layers for higher cardinality features. Also, creating interaction terms or polynomial features could be beneficial.
3.  **Regularization**: Implement L1/L2 regularization on the dense layers or add Dropout layers to prevent overfitting. High error rates can sometimes indicate that the model is overfitting to the training data.
4.  **Different Architectures**: Experiment with a different number of layers or more neurons per layer in the neural network. The current architecture might be too simple to capture the complexity of the data.
5.  **Learning Rate Schedule**: Utilize a learning rate scheduler instead of a fixed learning rate to adjust the learning rate dynamically during training, which can help optimize convergence and performance.
6.  **Outlier Treatment**: Re-examine the data for outliers in features or the target variable (`price`), as extreme values can heavily influence model training and evaluation metrics like MSE and RMSE.

## Summary:

### Q&A
*   **What was the impact of feature scaling on the model's performance?**
    Feature scaling using `MinMaxScaler` resulted in a slight reduction in the Root Mean Squared Error (RMSE) on the test data. The RMSE decreased from approximately 743601.44 (without scaling) to 740202.75 (with scaling), indicating a minor positive improvement.

*   **What are the suggested next steps if the error rate remains high?**
    Several strategies are suggested:
    1.  Conduct more Optuna trials to explore a wider range of hyperparameters.
    2.  Implement advanced feature engineering, including considering different encoding methods for categorical features or creating interaction/polynomial features.
    3.  Apply regularization techniques (L1/L2 or Dropout) to prevent overfitting.
    4.  Experiment with different neural network architectures (number of layers, neurons).
    5.  Utilize a learning rate schedule for dynamic learning rate adjustment during training.
    6.  Re-examine the data for outliers in features or the target variable.

### Data Analysis Key Findings
*   `MinMaxScaler` was successfully applied to the numerical training and testing features, resulting in `X_train_scaled` with shape (7665, 7) and `X_test_scaled` with shape (1917, 7).
*   The neural network model was retrained using the scaled data and the best hyperparameters found by Optuna: learning rate of 0.00148, 57 epochs, 53 nodes in the first dense layer, and 7 nodes in the second dense layer.
*   Evaluating the retrained model on the scaled test data yielded a Test Loss (MSE) of 547900896768.00 and a Test RMSE of 740202.75.
*   Feature scaling led to a marginal improvement in model performance, reducing the RMSE from 743601.44 (without scaling) to 740202.75 (with scaling).

### Insights or Next Steps
*   While feature scaling provided a slight improvement, the RMSE remains high, indicating that further model optimization and potentially more complex feature engineering are necessary to achieve a satisfactory performance.
*   The next steps should focus on more extensive hyperparameter tuning with Optuna, exploring different network architectures, applying regularization, and revisiting feature engineering techniques to better capture underlying data patterns.


### Retraining the Model with Best Parameters and Evaluating Performance

Now that Optuna has found the best hyperparameters, let's retrain a new model using these optimal settings on our training data. Then, we will evaluate its performance on the test set to get a final measure of how well it generalizes.

In [122]:
# Get the best parameters from the Optuna study
best_lr = new_study.best_params['trial']
best_epochs = new_study.best_params['echosBest']
best_n1 = new_study.best_params['n1Best']
best_n2 = new_study.best_params['n2Best']

print(f"Using Best Learning Rate: {best_lr}")
print(f"Using Best Epochs: {best_epochs}")
print(f"Using Best Layer 1 Nodes: {best_n1}")
print(f"Using Best Layer 2 Nodes: {best_n2}")

# Build the final model with the best parameters
final_model = tf.keras.Sequential([
    tf.keras.Input(shape=(7,)),
    tf.keras.layers.Dense(best_n1, activation='relu'),
    tf.keras.layers.Dense(best_n2, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model with the best learning rate if you want to use it with AdamW
# Note: AdamW usually integrates the learning rate into its optimizer,
# so 'lrBest' might be used to configure the AdamW optimizer itself.
# For simplicity, we'll keep the default AdamW learning rate unless specified otherwise.

final_model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=best_lr),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()],
                      loss='mse')

# Convert test data to float32
X_train_numeric = np.asarray(X_train, dtype=np.float32)
y_train_numeric = np.asarray(y_train, dtype=np.float32)
X_test_numeric = np.asarray(X_test, dtype=np.float32)
y_test_numeric = np.asarray(y_test, dtype=np.float32)

# Train the final model on the entire training set
print("\nTraining final model with best parameters...")
final_model.fit(X_train_numeric, y_train_numeric, epochs=best_epochs, verbose=1)

# Evaluate the final model on the test set
print("\nEvaluating final model on test data...")
loss, rmse = final_model.evaluate(X_test_numeric, y_test_numeric, verbose=2)
print(f"Test Loss (MSE): {loss:.2f}")
print(f"Test RMSE: {rmse:.2f}")

Using Best Learning Rate: 0.0001025804336194444
Using Best Epochs: 3
Using Best Layer 1 Nodes: 2
Using Best Layer 2 Nodes: 6

Training final model with best parameters...
Epoch 1/3
240/240 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 50743776116736.0000 - root_mean_squared_error: 6791864.0000
Epoch 2/3
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 34256690610176.0000 - root_mean_squared_error: 5526580.0000
Epoch 3/3
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16716481429504.0000 - root_mean_squared_error: 3883921.5000

Evaluating final model on test data...
60/60 - 0s - 4ms/step - loss: 602800521216.0000 - root_mean_squared_error: 776402.3125
Test Loss (MSE): 602800521216.00
Test RMSE: 776402.31


### Strategies to Address High Error Rate

If the error rate (RMSE) is still high after using the best parameters, here are some common strategies we can explore:

1.  **More Optuna Trials**: Currently, Optuna ran for 20 trials. Running it for more trials (e.g., 50, 100, or even more) might find a better combination of hyperparameters.
2.  **Feature Scaling**: Neural networks often perform better when input features are scaled (e.g., using `MinMaxScaler` or `StandardScaler`). Your features (like mileage, year, and ASCII sums) have very different ranges, which can make training difficult.
3.  **Advanced Feature Engineering**: Explore other ways to represent categorical features (e.g., one-hot encoding instead of ASCII sum, or embedding layers for high-cardinality categories). Also, creating new features from existing ones can be beneficial.
4.  **Regularization**: Add L1 or L2 regularization to Dense layers, or introduce Dropout layers to prevent overfitting, especially if the model performs well on training data but poorly on test data.
5.  **Different Architectures**: Experiment with more or fewer layers, or different numbers of nodes per layer.
6.  **Learning Rate Schedule**: Instead of a fixed learning rate, use a learning rate scheduler that adjusts the learning rate during training.
7.  **Data Augmentation (if applicable)**: Though less common for tabular data, if there are ways to create synthetic training examples, it could help.
8.  **Review Data Quality**: Double-check for outliers, incorrect data types, or missing values that might still be present and affecting model performance.

In [123]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Task
Apply `MinMaxScaler` to `X_train_numeric` and `X_test_numeric` to scale the features. Then, retrain the `final_model` using these scaled datasets and the best hyperparameters previously found by Optuna. Finally, evaluate the retrained model on the scaled test data and discuss the impact of feature scaling on the model's performance and suggest next steps if the error rate remains high.

## Apply Feature Scaling

### Subtask:
Apply `MinMaxScaler` to `X_train` and `X_test` to scale the features to a common range (0 to 1). This is crucial for neural networks to perform effectively with features having different scales.


**Reasoning**:
To apply feature scaling, I will import `MinMaxScaler`, instantiate it, fit it on the training data, and then transform both the training and testing data.



In [124]:
from sklearn.preprocessing import MinMaxScaler

# Instantiate MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on X_train_numeric and transform it
X_train_scaled = scaler.fit_transform(X_train_numeric)

# Transform X_test_numeric using the same fitted scaler
X_test_scaled = scaler.transform(X_test_numeric)

print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("MinMaxScaler applied successfully.")

X_train_scaled shape: (7665, 7)
X_test_scaled shape: (1917, 7)
MinMaxScaler applied successfully.


**Reasoning**:
Now that the features are scaled, I will retrain the `final_model` using the scaled training data (`X_train_scaled`) and the best hyperparameters found by Optuna. After retraining, I will evaluate its performance on the scaled test data (`X_test_scaled`) to observe the impact of feature scaling.



In [117]:
print(f"Using Best Learning Rate: {best_lr}")
print(f"Using Best Epochs: {best_epochs}")
print(f"Using Best Layer 1 Nodes: {best_n1}")
print(f"Using Best Layer 2 Nodes: {best_n2}")

# Build the final model with the best parameters
final_model = tf.keras.Sequential([
    tf.keras.Input(shape=(7,)),
    tf.keras.layers.Dense(best_n1, activation='relu'),
    tf.keras.layers.Dense(best_n2, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model with the best learning rate if you want to use it with AdamW
# Note: AdamW usually integrates the learning rate into its optimizer,
# so 'lrBest' might be used to configure the AdamW optimizer itself.
# For simplicity, we'll keep the default AdamW learning rate unless specified otherwise.

final_model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=best_lr),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()],
                      loss='mse')

# Train the final model on the scaled training set
print("\nTraining final model with best parameters and scaled data...")
final_model.fit(X_train_scaled, y_train_numeric, epochs=best_epochs, verbose=1)

# Evaluate the final model on the scaled test set
print("\nEvaluating final model on scaled test data...")
loss_scaled, rmse_scaled = final_model.evaluate(X_test_scaled, y_test_numeric, verbose=2)
print(f"Test Loss (MSE) with scaling: {loss_scaled:.2f}")
print(f"Test RMSE with scaling: {rmse_scaled:.2f}")

Using Best Learning Rate: 0.0014833843080737789
Using Best Epochs: 57
Using Best Layer 1 Nodes: 53
Using Best Layer 2 Nodes: 7

Training final model with best parameters and scaled data...
Epoch 1/57
240/240 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 13377141211136.0000 - root_mean_squared_error: 3402670.5000
Epoch 2/57
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4035989209088.0000 - root_mean_squared_error: 1575700.0000
Epoch 3/57
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1879162486784.0000 - root_mean_squared_error: 1182675.6250
Epoch 4/57
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7080730689536.0000 - root_mean_squared_error: 2550119.0000
Epoch 5/57
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7954795331584.0000 - root_mean_squared_error: 2524169.0000
Epoch 6/57
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6749687382016.0000 - root_mean_squared_error: 2463763.7500
Epoch 7/57
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 18289625399296.0000 - root_mean_squa

### Impact of Feature Scaling and Next Steps

After applying `MinMaxScaler` to the features and retraining the model:

*   **Previous RMSE (without scaling)**: 743601.44
*   **New RMSE (with scaling)**: 740202.75

**Impact of Feature Scaling:**
Feature scaling using `MinMaxScaler` resulted in a slight reduction in the Root Mean Squared Error (RMSE) from approximately 743601.44 to 740202.75. This indicates a minor improvement in the model's performance on the test data. While the change is not dramatic, it confirms that scaling the features to a common range can be beneficial for neural networks, which are sensitive to the scale of input data.

**Next Steps if Error Rate Remains High:**
Since the RMSE is still quite high, suggesting the model is not performing as accurately as desired, we should continue to explore further strategies from the list provided earlier:

1.  **More Optuna Trials**: The current Optuna study ran for only 5 trials. Increasing `n_trials` significantly (e.g., to 50, 100, or more) can help Optuna explore a wider range of hyperparameters and potentially find a much better combination.
2.  **Advanced Feature Engineering**: Revisit how categorical features (`model`, `transmission`, `fuel_type`) are encoded. Using a simple ASCII sum might not capture the underlying relationships effectively. Consider one-hot encoding for lower cardinality features or more sophisticated embedding layers for higher cardinality features. Also, creating interaction terms or polynomial features could be beneficial.
3.  **Regularization**: Implement L1/L2 regularization on the dense layers or add Dropout layers to prevent overfitting. High error rates can sometimes indicate that the model is overfitting to the training data.
4.  **Different Architectures**: Experiment with a different number of layers or more neurons per layer in the neural network. The current architecture might be too simple to capture the complexity of the data.
5.  **Learning Rate Schedule**: Utilize a learning rate scheduler instead of a fixed learning rate to adjust the learning rate dynamically during training, which can help optimize convergence and performance.
6.  **Outlier Treatment**: Re-examine the data for outliers in features or the target variable (`price`), as extreme values can heavily influence model training and evaluation metrics like MSE and RMSE.

## Summary:

### Q&A
*   **What was the impact of feature scaling on the model's performance?**
    Feature scaling using `MinMaxScaler` resulted in a slight reduction in the Root Mean Squared Error (RMSE) on the test data. The RMSE decreased from approximately 743601.44 (without scaling) to 740202.75 (with scaling), indicating a minor positive improvement.

*   **What are the suggested next steps if the error rate remains high?**
    Several strategies are suggested:
    1.  Conduct more Optuna trials to explore a wider range of hyperparameters.
    2.  Implement advanced feature engineering, including considering different encoding methods for categorical features or creating interaction/polynomial features.
    3.  Apply regularization techniques (L1/L2 or Dropout) to prevent overfitting.
    4.  Experiment with different neural network architectures (number of layers, neurons).
    5.  Utilize a learning rate schedule for dynamic learning rate adjustment during training.
    6.  Re-examine the data for outliers in features or the target variable.

### Data Analysis Key Findings
*   `MinMaxScaler` was successfully applied to the numerical training and testing features, resulting in `X_train_scaled` with shape (7665, 7) and `X_test_scaled` with shape (1917, 7).
*   The neural network model was retrained using the scaled data and the best hyperparameters found by Optuna: learning rate of 0.00148, 57 epochs, 53 nodes in the first dense layer, and 7 nodes in the second dense layer.
*   Evaluating the retrained model on the scaled test data yielded a Test Loss (MSE) of 547900896768.00 and a Test RMSE of 740202.75.
*   Feature scaling led to a marginal improvement in model performance, reducing the RMSE from 743601.44 (without scaling) to 740202.75 (with scaling).

### Insights or Next Steps
*   While feature scaling provided a slight improvement, the RMSE remains high, indicating that further model optimization and potentially more complex feature engineering are necessary to achieve a satisfactory performance.
*   The next steps should focus on more extensive hyperparameter tuning with Optuna, exploring different network architectures, applying regularization, and revisiting feature engineering techniques to better capture underlying data patterns.
